In [13]:
import pandas as pd
import re
import zipfile
import os

# Define o caminho para o arquivo ZIP
zip_path = "IT Bootcamp - Arquivos Case - Deloitte.zip"

# Extrai os arquivos do ZIP para a pasta "temp"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("temp")

# Lê os arquivos extraídos em DataFrames com codificação ajustada
usuarios = pd.read_csv(os.path.join("temp", "Lista de Usuários.txt"), sep="\t", encoding="latin1")
desligados = pd.read_csv(os.path.join("temp", "Colaboradores Desligados.txt"), sep="\t", encoding="latin1")
ativos = pd.read_csv(os.path.join("temp", "Colaboradores Ativos.txt"), sep="\t", encoding="latin1")

# Remove o prefixo de 2 caracteres na coluna "ID" em "usuarios"
usuarios['Matrícula'] = usuarios['ID'].apply(lambda x: re.sub(r'^[A-Za-z]{2}', '', str(x)))

# Converte as colunas de datas para o tipo datetime
usuarios['Data de Bloqueio'] = pd.to_datetime(usuarios['Data de Bloqueio'], format='%d/%m/%Y', errors='coerce')
usuarios['Data de Último Login'] = pd.to_datetime(usuarios['Data de Último Login'], format='%d/%m/%Y', errors='coerce')
desligados['Data Desligamento'] = pd.to_datetime(desligados['Data Desligamento'], format='%d/%m/%Y', errors='coerce')
ativos['Data de Admissão'] = pd.to_datetime(ativos['Data de Admissão'], format='%d/%m/%Y', errors='coerce')

# Converte a coluna "Matrícula" para string em todos os DataFrames
usuarios['Matrícula'] = usuarios['Matrícula'].astype(str)
desligados['Matrícula'] = desligados['Matrícula'].astype(str)
ativos['Matrícula'] = ativos['Matrícula'].astype(str)

# Filtra usuários ativos que não estão em "ativos" mas estão em "desligados"
status_ativo_lista_desligados = usuarios[
  (usuarios['Status'] == 'Ativo') &
  (~usuarios['Matrícula'].isin(ativos['Matrícula'])) &
  (usuarios['Matrícula'].isin(desligados['Matrícula']))
]

# Filtra usuários inativos que estão em "ativos" mas não estão em "desligados"
status_inativo_lista_ativos = usuarios[
  (usuarios['Status'] == 'Inativo') &
  (usuarios['Matrícula'].isin(ativos['Matrícula'])) &
  (~usuarios['Matrícula'].isin(desligados['Matrícula']))
]

# Concatena os dois DataFrames de inconsistências
unificado = pd.concat([status_ativo_lista_desligados, status_inativo_lista_ativos], ignore_index=True)

# Adiciona a data de desligamento ao DataFrame unificado
unificado = unificado.merge(
    desligados[['Matrícula', 'Data Desligamento']],
    on='Matrícula',
    how='left'
)

# Adiciona colunas indicando se a matrícula está em "ativos" ou "desligados"
unificado['Existe em Ativos'] = unificado['Matrícula'].isin(ativos['Matrícula'])
unificado['Existe em Desligados'] = unificado['Matrícula'].isin(desligados['Matrícula'])

# Define criticidade inicial como 1 para todas as inconsistências
unificado['Criticidade'] = 1

# Incrementa criticidade se o usuário está ativo indevidamente
unificado.loc[
    (unificado['Status'] == 'Ativo') & (~unificado['Existe em Ativos']),
    'Criticidade'
] += 1

# Incrementa criticidade se o usuário ativo indevidamente fez login após a data de desligamento
unificado.loc[
    (unificado['Status'] == 'Ativo') & (~unificado['Existe em Ativos']) & (unificado['Data de Último Login'] > unificado['Data Desligamento']),
    'Criticidade'
] += 1

# Ordena o DataFrame unificado por criticidade em ordem decrescente
unificado = unificado.sort_values(by='Criticidade', ascending=False)

# Salva o DataFrame unificado em um arquivo Excel
with pd.ExcelWriter("Relatório_Revogação_Acessos.xlsx") as writer:
    unificado.to_excel(writer, sheet_name="Inconsistências", index=False)

unificado


,ID,Módulo,Status,Data de Bloqueio,Data de Último Login,Matrícula,Data Desligamento,Existe em Ativos,Existe em Desligados,Criticidade
0,FC1881,RH (PONTO);COMPRAS,Ativo,NaT,2022-04-15,1881,2022-04-10,False,True,3
1,HM1484,RH (PONTO);FINANCEIRO,Ativo,NaT,2022-04-05,1484,2022-04-10,False,True,2
2,PP1834,RH (PONTO); RH (FOLHA PAGAMENTO),Ativo,NaT,2022-04-01,1834,2022-04-10,False,True,2
3,SP1225,RH (PONTO);TI,Inativo,2022-08-19,2022-08-16,1225,NaT,True,False,1
4,SR1238,RH (PONTO);CONTROLES INTERNOS;,Inativo,2022-08-31,2022-08-17,1238,NaT,True,False,1
